In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import time
from ta.volume import OnBalanceVolumeIndicator
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.momentum import PercentagePriceOscillator
from ta.momentum import ROCIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
#from ta.volume import SMAEaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import AccDistIndexIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.trend import VortexIndicator
from ta.trend import AroonIndicator
#from scipy.stats import percentileofscore
sip500=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
sip400=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0].iloc[:,1])
sip600=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1].iloc[:,1])
payload=np.append(sip500,sip400)
payload=np.append(payload,sip600)
payload=np.unique(payload)
payload=payload[payload!='BRK.B']
payload=payload[payload!='JW.A']
payload=payload[payload!='BF.B']
#print(payload)
df=pd.DataFrame()
df_out=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
today=datetime.today()
txt = "{:.2%}"
wind=14
for i in range(0,len(payload)):
    try:
        stk=payload[i]
        #df = yf.download(stk, start_date, end_date,interval=intr).dropna()
        #print(stk)
        dfd = yf.download(stk, start_date, today,interval='1d').dropna()
        dfd['Date'] = pd.to_datetime(dfd.index)
        dfd['Week_Number'] = dfd['Date'].dt.isocalendar().week
        dfd['Year'] = dfd['Date'].dt.isocalendar().year
        dfd['Close1']=dfd['Close'].shift(1)
        dfd['Volume1']=dfd['Volume'].shift(1)
        dfd['Low1']=dfd['Low'].shift(1)
        dfd['High1']=dfd['High'].shift(1)
        dfd['ChangePre']=dfd.Close-dfd.Close1
        dfd['Change']=dfd.Close-dfd.Open
        dfd['ChangePreP']=dfd.ChangePre/dfd.Close1
        dfd['ChangeP']=dfd.Change/dfd.Open
        dfd.set_index(['Year','Week_Number'],inplace=True)
        dfd['Mon']=dfd['Date'].dt.strftime('%m-%Y')


        dfw = dfd.groupby(['Year','Week_Number']).agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum'})
        dfw['Close1']=dfw['Close'].shift(1)
        dfw['Volume1']=dfw['Volume'].shift(1)
        dfw['Low1']=dfw['Low'].shift(1)
        dfw['High1']=dfw['High'].shift(1)

        dfd=pd.merge(left=dfd,right=dfw,how='left',left_index=True,right_index=True,suffixes=[None,'w'])
        #print(dfd.tail)
        dfm=dfd.resample('M', on='Date').agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum','Mon':'first'})
        dfm['Close1']=dfm['Close'].shift(1)
        dfm['Volume1']=dfm['Volume'].shift(1)
        dfm['Low1']=dfm['Low'].shift(1)
        dfm['High1']=dfm['High'].shift(1)

        dfd=pd.merge(left=dfd,right=dfm,how='left',left_on='Mon',right_on='Mon',suffixes=[None,'m'])
        dfd=dfd.dropna()
        dfd['Stock']=stk
        dfd.set_index('Stock',inplace=True)

        #dfd['fid']=ForceIndexIndicator(close=dfd.Close1,volume=dfd.Volume1).force_index()
        #dfd['fiw']=ForceIndexIndicator(close=dfd.Close1w,volume=dfd.Volume1w).force_index()
        #dfd['fim']=ForceIndexIndicator(close=dfd.Close1m,volume=dfd.Volume1m).force_index()
        #dfd['emd']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).ease_of_movement()
        #dfd['emw']=EaseOfMovementIndicator(high=dfd.High1w,low=dfd.Low1w,volume=dfd.Volume1w).ease_of_movement()
       # dfd['em_m']=EaseOfMovementIndicator(high=dfd.High1m,low=dfd.Low1m,volume=dfd.Volume1m).ease_of_movement()
        #dfd['semd']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).sma_ease_of_movement()
       # dfd['sem_w']=EaseOfMovementIndicator(high=dfd.High1w,low=dfd.Low1w,volume=dfd.Volume1w).sma_ease_of_movement()
        #dfd['sem_m']=EaseOfMovementIndicator(high=dfd.High1m,low=dfd.Low1m,volume=dfd.Volume1m).sma_ease_of_movement()
       # dfd['mac_d']=MACD(close=dfd.Close1).macd_diff()
       # dfd['mac_w']=MACD(close=dfd.Close1w).macd_diff()
       # dfd['mac_m']=MACD(close=dfd.Close1m).macd_diff()
        #dfd['dlr_d']=DailyLogReturnIndicator(close=dfd.Close1).daily_log_return() 
        dfd['ema_d']=EMAIndicator(close=dfd.Close1,window=14).ema_indicator()
        dfd['ema_w']=EMAIndicator(close=dfd.Close1w,window=14).ema_indicator()
        dfd['ema_m']=EMAIndicator(close=dfd.Close1m,window=14).ema_indicator()
       # dfd['bblid']=BollingerBands(close=dfd.Close1).bollinger_lband_indicator()
        dfd['adi_d']=AccDistIndexIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1,close=dfd.Close1).acc_dist_index()
       # dfd['obv_d']=OnBalanceVolumeIndicator(close=dfd.Close1,volume=dfd.Volume1).on_balance_volume()
        #dfd['dcl_w']=DonchianChannel(high=dfd.High1w,low=dfd.Low1w,close=dfd.Close1w).donchian_channel_lband() 
      #  dfd['dcl_m']=DonchianChannel(high=dfd.High1m,low=dfd.Low1m,close=dfd.Close1m).donchian_channel_lband() 
        #dfd['ppo_d']=PercentagePriceOscillator(close=dfd.Close1).ppo_hist()
        #dfd['vin_w']=VortexIndicator(high=dfd.High1w,low=dfd.Low1w,close=dfd.Close1w).vortex_indicator_neg()
        #dfd['aar_m']=AroonIndicator(close=dfd.Close1m).aroon_up()
       # dfd['roc_m']=ROCIndicator(close=dfd.Close1m).roc()
        dfd=dfd.dropna()
        #print(dfd.head)
        #x=1
        #x2=50
        #x3=50
        #print(dfd.columns)
        conditions = [ 
        (dfd['Open']>0)

        #& (dfd.fid>np.percentile(dfd.fid,100-x))
        #& (dfd.fiw>np.percentile(dfd.fiw,100-x2))
        #& (dfd.fim>np.percentile(dfd.fim,100-x3))
       # & (dfd.mcw>np.percentile(dfd.mcw,100-x))    
        #& (dfd.emd>np.percentile(dfd.emd,100-x))
        #& (dfd.emw>np.percentile(dfd.emw,100-x))
        #& (dfd.emm>np.percentile(dfd.emm,100-x))
        #& (dfd.semw>np.percentile(dfd.semw,100-x))
        #& (dfd.semm>np.percentile(dfd.semm,100-x))
        #&(dfd.dlr_d>np.percentile(dfd.dlr_d,100-x))

        &(dfd.adi_d>np.percentile(dfd.adi_d,99))    
        #&(dfd.obv_d>np.percentile(dfd.obv_d,99))

        #&(dfd.ppo_d>np.percentile(dfd.ppo_d,10))
        &(dfd.ema_d>np.percentile(dfd.ema_d,10))    


        #&(dfd.ema_d>np.percentile(dfd.ema_d,10))
        &(dfd.ema_w>np.percentile(dfd.ema_w,10))
       # &(dfd.vin_w<np.percentile(dfd.vin_w,90))

        #&(dfd.roc_m>np.percentile(dfd.roc_m,10))
        #&(dfd.aar_m>np.percentile(dfd.aar_m,10))
        &(dfd.ema_m>np.percentile(dfd.ema_m,10))
        #&(dfd.mac_w>np.percentile(dfd.mac_w,10))      
        #&(dfd.sem_w>np.percentile(dfd.sem_w,10))
        ,
        (dfd['Open']>0)
        #& (dfd.fid<np.percentile(dfd.fid,x))
        #& (dfd.fiw<np.percentile(dfd.fiw,x2))
       # & (dfd.fim<np.percentile(dfd.fim,x3))
        #& (dfd.mcw<np.percentile(dfd.mcw,x))
        #& (dfd.emd<np.percentile(dfd.emd,x))

        #& (dfd.emm<np.percentile(dfd.emm,x))
        #& (dfd.semw<np.percentile(dfd.semw,x))
        #& (dfd.semm<np.percentile(dfd.semm,x))
        #&(dfd.dlr_d<np.percentile(dfd.dlr_d,x))


       # &(dfd.obv_d<np.percentile(dfd.obv_d,1))

       # &(dfd.ppo_d<np.percentile(dfd.ppo_d,90))
        &(dfd.ema_d<np.percentile(dfd.ema_d,1))

        &(dfd.adi_d<np.percentile(dfd.adi_d,90))
        &(dfd.ema_w<np.percentile(dfd.ema_w,90))

        &(dfd.ema_m<np.percentile(dfd.ema_m,90))
       #&(dfd.vin_w>np.percentile(dfd.vin_w,10))    


        #&(dfd.dcl_w<np.percentile(dfd.dcl_w,90))
        #&(dfd.aar_m<np.percentile(dfd.aar_m,90))
        #&(dfd.dcl_m<np.percentile(dfd.dcl_m,90))
       # &(dfd.mac_w<np.percentile(dfd.mac_w,90))      
        #&(dfd.sem_w<np.percentile(dfd.sem_w,90))
        ]

        choices = [
        -1,1
        ]
        dfd['trade_pass']=np.select(conditions,choices,default=0)
        dfd_long=dfd[dfd['trade_pass']==1]
        dfd_short=dfd[dfd['trade_pass']==-1]
        df_out_l=pd.concat([df_out_l, dfd_long[dfd_long['Date']>='01-01-2021']])
        df_out_s=pd.concat([df_out_s, dfd_short[dfd_short['Date']>='01-01-2021']])
        dffl=df_out_l[['Date','Open','Close','Close1','Close1w','Close1m','adi_d','ema_d','ema_w','ema_m','Change','ChangePre']].copy()
        dffs=df_out_s[['Date','Open','Close','Close1','Close1w','Close1m','adi_d','ema_d','ema_w','ema_m','Change','ChangePre']].copy()
    except Exception as e: print(e)
print(dfff)
dfff.to_csv('trade_now.csv')
time.sleep(100000)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: 

NameError: name 'dfff' is not defined